# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [2]:
pprint(contracts)

Pretty printing has been turned OFF


In [31]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'custom_contracts_20201125111607.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [32]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [33]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [34]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [35]:
pprint(contracts)

{'addressProvider': {'SCORE': 'cx2a4267624dfeb9a79b931424fd8ac7b81c5cf805',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cx14ed4464849d2b34a43a54e4ae8bbdcf64bae671',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cx5dbae1f5dc20f2d0d16034a07869b6f0bedd14f6',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cxaa345a5dbea93b27754e44024d5ea14ccbf09c62',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cxb5083e61b035b31b0bffa7350ff276349069eb60',
                             'zip': '/home/elite/Desktop/OMM '
                                    'OG/openmoney

# Importing wallet

### Importing from private key

In [36]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [37]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

78703506361792600000000000

### Importing wallet from keystore file

In [9]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

Raised KeyStoreException while loading the wallet by the keystore file because the file is not found.
Traceback (most recent call last):
  File "/home/elite/.local/lib/python3.8/site-packages/iconsdk/wallet/wallet.py", line 101, in load
    with open(file_path, 'rb') as file:
FileNotFoundError: [Errno 2] No such file or directory: './keystores/keystore_test1.json'


KeyStoreException: File is not found. (Key store error)

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

In [20]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
#Run this when you need to zip the contract codes (before deploying them)
contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'sicx']
# for directory in {"score"}:
directory=os.getcwd()
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            # make_archive(archive_name, "zip", directory, file.name)
            contracts[file.name]['zip'] = archive_name 
contracts['oICX']['zip'] = directory + "/oToken"

contracts

{'oToken': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/oToken'},
 'sicx': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sicx'},
 'addressProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/addressProvider'},
 'lendingPoolDataProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPoolDataProvider'},
 'priceOracle': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/priceOracle'},
 'sample_token': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sample_token'},
 'lendingPoolCore': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPool': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPool'},
 'feeProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/feeProvider'},
 'oICX': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/oToken'}}

# Deploying all contracts


In [30]:
#this cell needs to run only when you need to deploy new set of contracts
for score, loc in contracts.items():
    params = {}
    print(score,loc)
    if score == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "oICX":
        params = {"_name":"BridgeICXInterestToken","_symbol":"oICX"}
    elif score == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

oToken {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/oToken', 'SCORE': 'cx3c3b8a599b0b90e8cf5f95ca390083c82a838c04'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
sicx {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sicx', 'SCORE': 'cxe2e5282c47555534c4d3cd9c922e459651db4b5b'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
addressProvider {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/addressProvider', 'SCORE': 'cxfbb26f333587547e2119459956c1a9bfa17c909c'}
lendingPoolDataProvider {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPoolDataProvider', 'SCORE': 'cx37d4865a0bc41e5f3

{'oToken': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/oToken',
  'SCORE': 'cxb7e290801d011961da1974dd84818082cf881246'},
 'sicx': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sicx',
  'SCORE': 'cxe74650f0903ef43102b74fb7d4949ad2f42eeff9'},
 'addressProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/addressProvider',
  'SCORE': 'cx2a4267624dfeb9a79b931424fd8ac7b81c5cf805'},
 'lendingPoolDataProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPoolDataProvider',
  'SCORE': 'cxb5083e61b035b31b0bffa7350ff276349069eb60'},
 'priceOracle': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/priceOracle',
  'SCORE': 'cx790de19720a6af19c50d264f53529bab60fe6837'},
 'sample_token': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sample_token',
  'SCORE': 'cx629b3f914cb8202745ff6da14d69f1aee1087cc1'},
 'lendingPoolCore': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/l

In [46]:
#use this to update a pre existing pkl file
filename = 'updated20201125_' + contracts_file_custom
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as updated20201125_custom_contracts_20201122161857.pkl


{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider',
  'SCORE': 'cx726e8bacc442256099dda50438c11d004215afc5'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool',
  'SCORE': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider',
  'SCORE': 'cxe408b46a010ff5f82e9e45610a0aadb4ff2b073c'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore',
  'SCORE': 'cx3daa5c2517fdd44bdf37904e5e537eefafb4d69d'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken',
  'SCORE': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle',
  'SCORE': 'cx6703fdc7e0e65c5195d6dbfb59a838f76a9c19b1'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token'

In [38]:
pprint(contracts)
print(contracts.items())

{'addressProvider': {'SCORE': 'cx2a4267624dfeb9a79b931424fd8ac7b81c5cf805',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cx14ed4464849d2b34a43a54e4ae8bbdcf64bae671',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cx5dbae1f5dc20f2d0d16034a07869b6f0bedd14f6',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cxaa345a5dbea93b27754e44024d5ea14ccbf09c62',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cxb5083e61b035b31b0bffa7350ff276349069eb60',
                             'zip': '/home/elite/Desktop/OMM '
                                    'OG/openmoney

# Deploying otoken and sicx for icx

In [31]:
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX']["SCORE"] = res.get('scoreAddress', '')
contracts['oICX']["zip"] = '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken'

params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['sicx']["SCORE"] = res.get('scoreAddress', '')
contracts['sicx']["zip"] = '/home/shubham/Work/ICON/openmoneymarket-mono/score/sicx'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xb1ae2334359e4fb41741ac4bbf66cc9166ee4daeafac8fb2885d800fa62c554c', 'blockHeight': 1128147, 'blockHash': '0xc836630e1d11925b0b4286f38e469d779301c15aa98994c5d795070aa1955a44', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx6dbb85df52d7b12d2c00d673d89c5bf21ee29be2', 'stepUsed': 1144284440, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1144284440, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [57]:
pprint(contracts)

{'addressProvider': {'SCORE': 'cxed440f1da2cf3087ac5ef12217a13f6321de0375',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cxede316aa65b7df13633599e67490daf1fd32b8a5',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
                             'zip': '/home/elite/Desktop/OMM '
                                    'OG/openmoney

# Updating a contract

In [33]:
contract = 'lendingPool'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

ValueError: Invalid path /home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool

# Configuring SCOREs's parameters

In [39]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setUSDbAddress', 'params':{'_address': contracts['sample_token']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 25*10**14}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token']['SCORE'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']['SCORE']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']['SCORE']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"1{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"5{'0'*17}"} ]}}]

In [40]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

0569fdedc8baa33ef424773e53700dc09c9d0e4fee230a86e9a159b6', 'blockHeight': 1132418, 'blockHash': '0x6934c8ab479b2b01614c5eaf36db63e65092401b31d65d510214e7a3a1fac510', 'txIndex': 1, 'to': 'cxb5083e61b035b31b0bffa7350ff276349069eb60', 'stepUsed': 152720, 'stepPrice': 10000000000, 'cumulativeStepUsed': 152720, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Additional setup for icx reserve

In [41]:
icx_additional=[{'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
                {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'sICX','_quote':'USD','_rate':5*10**17}},
                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx']['SCORE'],'_sym':"sICX"}},
                {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']['SCORE']}},
                {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']['SCORE']}},
                {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}},
                 
                {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
                {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']['SCORE']}},
                {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
                {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},


]

In [42]:
for sett in icx_additional:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setSICXAddress, with parameters {'_address': 'cxe74650f0903ef43102b74fb7d4949ad2f42eeff9'} on the lendingPool contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x7b99a17e27e933994019e233d8c0878f4e19914e740b60cc68e23650e529d696', 'blockHeight': 1132437, 'blockHash': '0x810d652680a7a5eaf864e7df28c74c306e10efeb4b5620f348dce3744a374ba2', 'txIndex': 1, 'to': 'cx5dbae1f5dc20f2d0d16034a07869b6f0bedd14f6', 'stepUsed': 150520, 'stepPrice': 10000000000, 'cumulativeStepUsed': 150520, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Adding USDb reserve in LendingPoolCore



In [43]:
params ={"_reserve": {"reserveAddress":contracts['sample_token']['SCORE'],"oTokenAddress":contracts['oToken']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"600000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x37cb74c1a407726d132e25ed83696a50f40accc5324f79aab85a61d47a4e568d',
 'blockHeight': 1132466,
 'blockHash': '0x1c341dc542a0db9423cbc18311217968cd0e33c046f99eeb17fa47547580508f',
 'txIndex': 1,
 'to': 'cxaa345a5dbea93b27754e44024d5ea14ccbf09c62',
 'stepUsed': 271600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 271600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding ICX reserve

In [44]:
params ={"_reserve": {"reserveAddress":contracts['sicx']['SCORE'],"oTokenAddress":contracts['oICX']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"330000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x0d62737848162cad4da223d5f2e4b855257ed77b0ae7d775b4bdc874d48cdbb6',
 'blockHeight': 1132470,
 'blockHash': '0xaadb3c095047f34daae1d0fc74307c98ec70608bd6055a30e2e6c1cd4fa9ba01',
 'txIndex': 1,
 'to': 'cxaa345a5dbea93b27754e44024d5ea14ccbf09c62',
 'stepUsed': 271360,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 271360,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Transfering USDb to a wallet

In [45]:
params = {"_to": "hx91bf040426f226b3bfcd2f0b5967bbb0320525ce", "_value": 1000000000000000000000,}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x893d8e2f08a5d3c801c94d361099d8bab16c2e7e071e5c04616b7e89d466c63d',
 'blockHeight': 1132480,
 'blockHash': '0xd82a6eddf21d8f92c75a87fa9620193067898d121bc88ab97577ac71bce3eaa8',
 'txIndex': 1,
 'to': 'cx629b3f914cb8202745ff6da14d69f1aee1087cc1',
 'stepUsed': 158920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 158920,
 'eventLogs': [{'scoreAddress': 'cx629b3f914cb8202745ff6da14d69f1aee1087cc1',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x3635c9adc5dea00000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

# Depositing USDb from a wallet

In [68]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x731107653cf36b689816dd3305c33739fd893840f9da89c4a03cfc4e0c829a46', 'blockHeight': 9633486, 'blockHash': '0x4c23fbf9e810346acfd66ea9117049fcbdea42652289cd7f6331eb9d98395dac', 'txIndex': 2, 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025', 'stepUsed': 833680, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1024680, 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b', 'indexed': ['PrintData(str,int,int,int)', 'params check core line 408', '0x3635c9adc5dea00000', '0x0'], 'data': ['0x0']}, {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b', 'indexed': ['PrintData(str,int,int,int)', 'rate value core line 412', '0x7792aa37489b', '0xeaaa48e230926'], 'data': ['0x0']}, {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b', 'indexed': ['ReserveUpdated(Address,int,int,int,int)', 'cx19be1be7b4e7750863edf966faac3f42dec21025', '0x7792aa37489b', '0xeaaa48e230926'], 'data': ['0xde0b74e7acd5c12', '0xde0d3bd9dd34a0d']}, {'scoreAddress': 'cx425846

# Borrowing USDb

In [43]:
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d', 'stepLimit': '0x14bbf8', 'timestamp': '0x5b46dd9bf37cd', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx1592d568457a2881e4bd36117a8295ac2a000377', '_amount': '0x8ac7230489e80000'}}, 'signature': 'Folqgh9js3lLO0q/dtvGledD0lxOz/LirI6qn2i7J4AWTIR7p2T4cxqoKL4lfDXYE9q5cEPlAjdSF6pWXzwJlgA='}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x8dd5715179fb035028e00749c33effa1d3efd06c4955ec71504df6cc145dd53b',
 'blockHeight': 4799,
 'blockHash': '0xbce5c543a5e8b5bd12ed8adf8606a2b135bbe0638fd69bfa816dcddc3651ead2',
 'txIndex': 1,
 'to': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
 'stepUsed': 1258840,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1258840,
 'eventLogs': [{'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'available liquidity at pool line 245',
    '0x2e141ea081ca080000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'data at pool line 256',
    '0x3635dd4440d4c9f9a8',
    '0x8274de6e63dbd6b43'],
   'data': ['0x0']},
  {'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'amout of collateral needed USD pool line 268',
    '0x2519f35d8353d2db',
    '0x0'],
   'data': ['0x0']},

# Reedeming/Withdrawing USDb

In [ ]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

In [34]:
params ={"_reserveAddress": contracts['sample_token']['SCORE'],"_baseLTVasCollateral" :33*10**16}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("updateBaseLTVasCollateral")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx327a386a9f8076045679fc132a8baa9e270ca43b', 'stepLimit': '0x3ea80', 'timestamp': '0x5b4d764400c45', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateBaseLTVasCollateral', 'params': {'_reserveAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025', '_baseLTVasCollateral': '0x494654067e10000'}}, 'signature': 'g7dSUkTfVM+/zeivI3GhttfsbcabIPfbh6ZIJAm+ChEBJ8E8XhbxDdKzf1p2wzyG49tYwISwN7d3G6redmkj9wE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x84c2bc5d2ef49ac9d74f19a54f43dcd213c10e9e5772c11e2be9ff343133df9c',
 'blockHeight': 9594840,
 'blockHash': '0xfb4f010715f54f4a8cc051bf40065e7c388d49af1d35099007e5cc779f515f9d',
 'txIndex': 4,
 'to': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
 'stepUsed': 156640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 959180,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Repaying USDb

In [38]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 10000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 10000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xa00f366c3e3bc59fa930e6c4134a4de0ab0437250175baf3a3342ac194d7049a',
 'blockHeight': 2668,
 'blockHash': '0x51bc669d21d2e946367aea11c77091441d579d42044fe1abaf3842a27ded2a41',
 'txIndex': 1,
 'to': 'cx1592d568457a2881e4bd36117a8295ac2a000377',
 'stepUsed': 951460,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 951460,
 'eventLogs': [{'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x853a0d2313c00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 593',
    '0x2e0e918aa053e00000',
    '0x82738b02dc459515c'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'Utilization rate core line 598',
    '0x216541f7accc0ec',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd

In [23]:
print(icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"))

9450000385630198414715


# Depositing ICX from Wallet

In [14]:
params = {"_amount": 500 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']['SCORE']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(500 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


{'txHash': '0x6898fffb800cbbfb62fa614e96771389a74e8127ef2fe46baf8d2a419d979472',
 'blockHeight': 9634462,
 'blockHash': '0x8f9d7cc0a52b0ac173af79dff48a43e2690e37d814b470f5ca250f89ee1f72e5',
 'txIndex': 1,
 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
 'stepUsed': 633520,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 633520,
 'eventLogs': [{'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
    'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
   'indexed': ['Mint(Address,int)',
    'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x1b1ae4d6e2ef500000',
    '0x0'],
   'data'

# Withdrawing ICX 

In [21]:
params = {"_amount": 250 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX']['SCORE'])\
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("redeem") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)
        

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x666b160e4f3cb5584a71b4d898864fa186c769a1e066d92bf71f9b24eb5566f3',
 'blockHeight': 9634974,
 'blockHash': '0x06d366c10c708bfc4f3d5bc8697b8fd862368d8e410c6d11835c8c17776ca064',
 'txIndex': 1,
 'to': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
 'stepUsed': 1322300,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1322300,
 'eventLogs': [{'scoreAddress': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0xd8d726b7177a80000']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x0',
    '0xd8d726b7177a80000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['P

# Readonly methods to get data

### Getting all the addresses

In [37]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['addressProvider']['SCORE'])\
    .method("getAllAddresses")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'collateral': {'USDb': 'cx134a1a0309aaf88a00ad1fee4983b7ed43454ab6',
                'sICX': 'cx735ee7e87d30d76a83b27c7bc62a84b5beafa34b'},
 'oTokens': {'oICX': 'cx6dbb85df52d7b12d2c00d673d89c5bf21ee29be2',
             'oUSDb': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe'},
 'systemContract': {'LendingPool': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48',
                    'LendingPoolDataProvider': 'cx726e8bacc442256099dda50438c11d004215afc5'}}


### Getting specific reserve data

In [38]:
#this returns the reserve data of USDb
params = {'_reserve': contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': '0x0',
 'baseLTVasCollateral': 0.33,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': '0x0',
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': '0x0',
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': '0x0',
 'oTokenAddress': 'cx6dbb85df52d7b12d2c00d673d89c5bf21ee29be2',
 'reserveAddress': 'cx735ee7e87d30d76a83b27c7bc62a84b5beafa34b',
 'totalBorrows': '0x0',
 'totalLiquidity': '0x0',
 'usageAsCollateralEnabled': '0x1'}


# Reserve Account Data

In [39]:
#this returns the reserve data of USDb

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveAccountData")\
    .build()
response = icon_service.call(_call)
# print(response)
for key,values in response.items():
    response[key]=int(values,0)/10**18
        

pprint(response)

{'availableLiquidityBalanceUSD': 298500.0,
 'totalBorrowsBalanceUSD': 501500.1141552619,
 'totalCollateralBalanceUSD': 800000.1141552619,
 'totalLiquidityBalanceUSD': 800000.1141552619}


### Getting data of all reserves

In [43]:
#this returns the reserve data of all the reserves
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': 298500.0,
          'baseLTVasCollateral': 0.6,
          'borrowCumulativeIndex': 1.0000011584585715,
          'borrowRate': 0.08359381655340004,
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': 1606110048844793,
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 0.65,
          'liquidityCumulativeIndex': 1.0000005791745306,
          'liquidityRate': 0.04716259038240734,
          'oTokenAddress': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe',
          'reserveAddress': 'cx134a1a0309aaf88a00ad1fee4983b7ed43454ab6',
          'totalBorrows': 501500.1141552619,
          'totalLiquidity': 800000.1141552619,
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.0,
          'borrowRate': '0x0',
          'borrowingEnabl

### Getting user account data

In [47]:
#returns the user data of all the reserves combined
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 0.65,
 'currentLtv': 0.33,
 'healthFactor': -1e-18,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 5000.003205614853,
 'totalFeesUSD': 0.0,
 'totalLiquidityBalanceUSD': 5000.003205614853}


In [49]:
#returns the user data of all the reserves combined
params = {'_reserve': contracts['sample_token']['SCORE'],'_amount':10*10**18,"_fee":1,'_userCurrentBorrowBalanceUSD':0,'_userCurrentFeesUSD':0,'_userCurrentLtv':5*10**17}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("calculateCollateralNeededUSD")\
    .params(params)\
    .build()
response = icon_service.call(_call)
response = int(response,0)/10**18
# for key,values in response.items():
#     if key!="lastUpdateTimestamp":
#         response[key]=int(values,0)/10**18
#     else:
#         response[key]=int(values,0)
print(response)

20.0


### Getting user reserve data

In [52]:
#returns the data of a specific user for a specific reserve
#this returns the data of user for USDb reserve
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.0,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 0.0,
 'currentOTokenBalanceUSD': 0.0,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 0.0,
 'userBorrowCumulativeIndex': 0.0}


# Getting All Reserve Data

In [44]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'USDb': {'availableLiquidity': '0x3f35b7a91f3815900000',
          'baseLTVasCollateral': '0x853a0d2313c0000',
          'borrowCumulativeIndex': '0xde0b7c160e6fb0d',
          'borrowRate': '0x128fc221d15a6e5',
          'borrowingEnabled': '0x1',
          'decimals': '0x12',
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': '0x5b4bfa2ee7bf9',
          'liquidationBonus': '0xa',
          'liquidationThreshold': '0x905438e60010000',
          'liquidityCumulativeIndex': '0xde0b73a80e1fe23',
          'liquidityRate': '0xa78e20466e3ea9',
          'oTokenAddress': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe',
          'reserveAddress': 'cx134a1a0309aaf88a00ad1fee4983b7ed43454ab6',
          'totalBorrows': '0x6a32602b7ababa00e840',
          'totalLiquidity': '0xa96817d499f2cf90e840',
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': '0x494654067e10000',
          'borrowCu

### Getting the data of user for all reserves

In [45]:
#returns the data of a specific user for all reserves

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.08359381655340004,
          'currentBorrowBalance': 501746.8555771935,
          'currentBorrowBalanceUSD': 501746.8555771935,
          'currentOTokenBalance': 800222.1155262836,
          'currentOTokenBalanceUSD': 800222.1155262836,
          'lastUpdateTimestamp': 1606109390013725,
          'liquidityRate': 0.04716259038240734,
          'originationFee': 0.0,
          'principalBorrowBalance': 501500.1141552619,
          'principalBorrowBalanceUSD': 501500.1141552619,
          'useAsCollateral': 1e-18,
          'userBorrowCumulativeIndex': 1.0000001902587698},
 'sICX': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCol

### Get reserve configurations


In [61]:
#returns the configurations of a specific reserve
params = {"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 330000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [62]:
#returns the configurations of all the   reserves

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1},
 'icx': {'baseLTVasCollateral': 330000000000000000,
         'borrowingEnabled': 1,
         'decimals': 18,
         'isActive': 1,
         'liquidationBonus': 10,
         'liquidationThreshold': 650000000000000000,
         'usageAsCollateralEnabled': 1}}


## Realtime USDb balance of user(with accrued interest)

In [33]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


In [20]:

# params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getOracleAddress")\
    .build()
response = icon_service.call(_call)
print(response)

cx8397f621adac63d786c24cfdf920e683ae320dfd


In [ ]:
995000000570919367819
995000000564440311889
894000001574533095848
894000000615847612243
894000001574533095848
795000001589272371715
795000001731223349511